# Unsafe Code in Rust

### Introduction to Unsafe Code

In [3]:
let safe_variable = 42;

println!("This is safe code: {}", safe_variable);

unsafe {
     let raw_pointer: *const i32 = &safe_variable;
     println!("Unsafe raw pointer value: {}", *raw_pointer);
}

This is safe code: 42
Unsafe raw pointer value: 42


()

### Balancing Performance and Safety Considerations

In [5]:
let data = vec![1, 2, 3, 4, 5];

for &value in &data {
    println!("Value: {}", value);
}

unsafe {
    let raw_pointer = data.as_ptr();
    for i in 0..data.len() {
        println!("Value at index {}: {}", i, *raw_pointer.add(i));
    }
}

Value: 1
Value: 2
Value: 3
Value: 4
Value: 5
Value at index 0: 1
Value at index 1: 2
Value at index 2: 3
Value at index 3: 4
Value at index 4: 5


()

## Real-World Examples

### Example 1: Database Interaction using FFI

In [7]:
:dep rusqlite = {version="0.30.0"}

In [8]:
unsafe extern "C" fn select_data_callback(
    data: *mut std::ffi::c_void,
    columns: i32,
    values: *mut *mut i8,
    _names: *mut *mut i8,
) -> i32 {
    let data = data as *mut Vec<Vec<String>>;

    let mut row_data = Vec::new();
    for i in 0..columns {
        let value = std::ffi::CStr::from_ptr(*values.offset(i as isize)).to_string_lossy();
        row_data.push(value.into_owned());
    }

    println!("Row Data in Callback: {:?}", row_data);

    let data = &mut *data;
    data.push(row_data);

    0 // Continue processing rows
}

fn advanced_database_interaction() {
    unsafe {
        let mut db: *mut rusqlite::ffi::sqlite3 = std::ptr::null_mut();
        let result = rusqlite::ffi::sqlite3_open(":memory:\0".as_ptr() as *const i8, &mut db);
        println!("SQLite database connection result: {}", result);
        println!("SQLite database pointer: {:p}", db);

        if result != rusqlite::ffi::SQLITE_OK {
            println!("Error opening SQLite database");
            return;
        }

        let create_table_query =
            "CREATE TABLE IF NOT EXISTS users (id INTEGER PRIMARY KEY, name TEXT);\0";
        let create_table_query_ptr = create_table_query.as_ptr() as *const i8;
        let mut create_table_err_msg: *mut i8 = std::ptr::null_mut();
        let create_table_result = rusqlite::ffi::sqlite3_exec(
            db,
            create_table_query_ptr,
            None,
            std::ptr::null_mut(),
            &mut create_table_err_msg,
        );

        println!("Create table result: {}", create_table_result);
        println!(
            "Create table error message pointer: {:p}",
            create_table_err_msg
        );

        if create_table_result != rusqlite::ffi::SQLITE_OK {
            println!("Error creating table: {:?}", create_table_err_msg);
            rusqlite::ffi::sqlite3_free(create_table_err_msg as *mut std::ffi::c_void);
            return;
        }

        let insert_data_query = "INSERT INTO users (name) VALUES ('Mahmoud Harmouch');\0";
        let insert_data_query_ptr = insert_data_query.as_ptr() as *const i8;
        let mut insert_data_err_msg: *mut i8 = std::ptr::null_mut();
        let insert_data_result = rusqlite::ffi::sqlite3_exec(
            db,
            insert_data_query_ptr,
            None,
            std::ptr::null_mut(),
            &mut insert_data_err_msg,
        );

        println!("Insert data result: {}", insert_data_result);
        println!(
            "Insert data error message pointer: {:p}",
            insert_data_err_msg
        );

        if insert_data_result != rusqlite::ffi::SQLITE_OK {
            println!("Error inserting data: {:?}", insert_data_err_msg);
            rusqlite::ffi::sqlite3_free(insert_data_err_msg as *mut std::ffi::c_void);
            return;
        }

        let select_data_query = "SELECT * FROM users;\0";
        let select_data_query_ptr = select_data_query.as_ptr() as *const i8;
        let mut select_data_err_msg: *mut i8 = std::ptr::null_mut();
        let mut select_data_result: Vec<Vec<String>> = Vec::new();
        let select_data_callback = Some(
            select_data_callback
                as unsafe extern "C" fn(
                    *mut std::ffi::c_void,
                    i32,
                    *mut *mut i8,
                    *mut *mut i8,
                ) -> i32,
        );
        let data_ptr = &mut select_data_result as *mut Vec<Vec<String>> as *mut std::ffi::c_void;
        let select_data_result = rusqlite::ffi::sqlite3_exec(
            db,
            select_data_query_ptr,
            select_data_callback,
            data_ptr,
            &mut select_data_err_msg,
        );

        println!("Select data result: {}", select_data_result);
        println!(
            "Select data error message pointer: {:p}",
            select_data_err_msg
        );

        if select_data_result != rusqlite::ffi::SQLITE_OK {
            println!("Error selecting data: {:?}", select_data_err_msg);
            rusqlite::ffi::sqlite3_free(select_data_err_msg as *mut std::ffi::c_void);
            return;
        }

        rusqlite::ffi::sqlite3_close(db);
        println!("SQLite database connection closed");
    }
}

advanced_database_interaction()

SQLite database connection result: 0
SQLite database pointer: 0x555f4ab00fe8
Create table result: 0
Create table error message pointer: 0x0
Insert data result: 0
Insert data error message pointer: 0x0
Row Data in Callback: ["1", "Mahmoud Harmouch"]
Select data result: 0
Select data error message pointer: 0x0
SQLite database connection closed


()

### Example 2: Advanced Image Processing with OpenCV

In [9]:
:dep opencv = {version="0.88.1"}

In [10]:
extern crate opencv;

use opencv::prelude::MatTraitConst;
use opencv::core::{BORDER_DEFAULT, CV_8UC3};
use std::ffi::c_void;

fn image_processing() {
    unsafe {
        // Create sample data
        let sample_data: Vec<u8> = vec![0; 480 * 640 * 3]; // Assuming 3 channels

        let mut image: opencv::core::Mat = opencv::core::Mat::new_rows_cols_with_data(
            480, 640, CV_8UC3, sample_data.as_ptr() as *mut c_void, 0,
        ).unwrap();

        println!(
            "Original Image: Rows={}, Cols={}, Channels={}",
            image.rows(),
            image.cols(),
            image.channels()
        );

        let mut blurred_image = opencv::core::Mat::default();
        opencv::imgproc::blur(
            &image,
            &mut blurred_image,
            opencv::core::Size::new(5, 5),
            opencv::core::Point::new(2, 2),
            BORDER_DEFAULT,
        )
        .unwrap();

        // Move the temporary variable back to the original variable
        std::mem::swap(&mut image, &mut blurred_image);

        println!(
            "Image after Gaussian Blur: Rows={}, Cols={}, Channels={}",
            image.rows(),
            image.cols(),
            image.channels()
        );

        let mut gray_image = opencv::core::Mat::default();
        opencv::imgproc::cvt_color(
            &image,
            &mut gray_image,
            opencv::imgproc::COLOR_BGR2GRAY,
            0,
        )
        .unwrap();

        // Move the temporary variable back to the original variable
        std::mem::swap(&mut image, &mut gray_image);

        println!(
            "Image after Grayscale Conversion: Rows={}, Cols={}, Channels={}",
            image.rows(),
            image.cols(),
            image.channels()
        );

        // Canny edge detection with a temporary variable
        let mut canny_image = opencv::core::Mat::default();
        opencv::imgproc::canny(
            &image,
            &mut canny_image,
            50.0,
            150.0,
            3,
            false,
        )
        .unwrap();

        // Move the temporary variable back to the original variable
        std::mem::swap(&mut image, &mut canny_image);

        println!(
            "Image after Canny Edge Detection: Rows={}, Cols={}, Channels={}",
            image.rows(),
            image.cols(),
            image.channels()
        );
    }
}

image_processing()

Original Image: Rows=480, Cols=640, Channels=3
Image after Gaussian Blur: Rows=480, Cols=640, Channels=3
Image after Grayscale Conversion: Rows=480, Cols=640, Channels=1
Image after Canny Edge Detection: Rows=480, Cols=640, Channels=1


()

### Example 3: Custom Memory Management in a Data Processing Pipeline

In [11]:
struct CustomMemoryManager {
    allocate: unsafe extern "C" fn(size: usize) -> *mut u8,
    deallocate: unsafe extern "C" fn(ptr: *mut u8, size: usize),
}

fn process_data_with_custom_memory_manager(data: &[u8], memory_manager: &CustomMemoryManager) {
    unsafe {
        let buffer_size = data.len() * 2;
        let custom_buffer = (memory_manager.allocate)(buffer_size);

        println!("Allocated {} bytes at {:p}", buffer_size, custom_buffer);

        std::ptr::copy_nonoverlapping(data.as_ptr(), custom_buffer, data.len());

        // Additional data processing using the custom buffer would happen here

        (memory_manager.deallocate)(custom_buffer, buffer_size);

        println!("Deallocated {} bytes at {:p}", buffer_size, custom_buffer);
    }
}

unsafe extern "C" fn custom_allocate(size: usize) -> *mut u8 {
    let ptr = std::alloc::alloc(std::alloc::Layout::from_size_align(size, 1).unwrap());
    println!("Allocated {} bytes at {:p}", size, ptr);
    ptr
}

unsafe extern "C" fn custom_deallocate(ptr: *mut u8, size: usize) {
    println!("Deallocating {} bytes at {:p}", size, ptr);
    std::alloc::dealloc(ptr, std::alloc::Layout::from_size_align(size, 1).unwrap());
}

let custom_memory_manager = CustomMemoryManager {
    allocate: custom_allocate,
    deallocate: custom_deallocate,
};
let input_data = vec![1, 2, 3, 4, 5];
process_data_with_custom_memory_manager(&input_data, &custom_memory_manager);

Allocated 10 bytes at 0x555f4ab08790
Allocated 10 bytes at 0x555f4ab08790
Deallocating 10 bytes at 0x555f4ab08790
Deallocated 10 bytes at 0x555f4ab08790


### Best Practices for Using Unsafe Code

In [12]:
mod unsafe_module {
    // Safe Rust function
    pub fn safe_function(data: &mut Vec<i32>) {
        // Safe Rust code
        data.push(42);
        println!("Safe function: {:?}", data);
    }

    // Unsafe Rust function
    pub unsafe fn unsafe_function(data: &mut Vec<i32>) {
        // Unsafe code encapsulated within a safe abstraction
        let reference = data.as_mut_ptr();
        *reference.offset(0) = 10;
        println!("Unsafe function: {:?}", data);
    }
}

let mut data = vec![1, 2, 3, 4, 5];

// Calling the safe function
unsafe_module::safe_function(&mut data);

// Calling the unsafe function within a safe context
unsafe {
    unsafe_module::unsafe_function(&mut data);
}

// Accessing the modified data after calling the unsafe function
println!("Main function: {:?}", data);

Safe function: [1, 2, 3, 4, 5, 42]
Unsafe function: [10, 2, 3, 4, 5, 42]
Main function: [10, 2, 3, 4, 5, 42]


### Memory Safety Violations in Unsafe Code

In [13]:
let mut data = vec![1, 2, 3, 4, 5];

let reference = data.as_mut_ptr();

unsafe {
    *reference.offset(0) = 10;
}

println!("Modified data: {:?}", data);

Modified data: [10, 2, 3, 4, 5]


In [14]:
let mut data = vec![1, 2, 3, 4, 5];

for element in data.iter_mut() {
    *element = *element + 10;
}

println!("Modified data: {:?}", data);

Modified data: [11, 12, 13, 14, 15]


### Case Studies Highlighting Real-World Consequences

### Case Study 1: Heartbleed Vulnerability

In [15]:
use std::collections::HashMap;

struct Server {
    user_data: HashMap<String, String>,
}

impl Server {
    fn get_user_data(&self, username: &str) -> Option<&String> {
        self.user_data.get(username)
    }

    unsafe fn unsafe_heartbleed(&self) {
        let buffer: Vec<u8> = self
            .user_data
            .values()
            .flat_map(|s| s.as_bytes().to_vec())
            .collect();

        let mut response = Vec::new();
        response.resize(buffer.len(), 0);
        response.copy_from_slice(&buffer);

        send_response(response);
    }
}

fn send_response(response: Vec<u8>) {

    let response_str = String::from_utf8_lossy(&response);

    println!("Sending response: {}", response_str);
}

let server = Server {
    user_data: [
        ("Mahmoud".to_string(), "password123".to_string()),
        ("Prime".to_string(), "secret456".to_string()),
    ]
    .iter()
    .cloned()
    .collect(),
};

unsafe {
    server.unsafe_heartbleed();
}

Sending response: password123secret456


()

### Case Study 2: Ariane 5 Flight 501 Failure

In [16]:
use std::sync::atomic::{AtomicBool, AtomicUsize, Ordering};
use std::sync::Arc;
use std::thread;

struct GuidanceSystem {
    safe_mode: AtomicBool,
    unsafe_counter: AtomicUsize,
}

impl GuidanceSystem {
    fn safe_critical_calculation(&self, payload_mass: f64) -> i32 {
        if self.safe_mode.load(Ordering::SeqCst) {
            let result = (32767.0 * 9.81 / payload_mass).round();

            if result >= i32::MIN as f64 && result <= i32::MAX as f64 {
                result as i32
            } else {
                if result > 0.0 {
                    i32::MAX
                } else {
                    i32::MIN
                }
            }
        } else {
            (32767.0 * 9.81 / payload_mass) as i32
        }
    }

    unsafe fn unsafe_use_result(&self, result: i32) {
        self.unsafe_counter
            .fetch_add(result as usize, Ordering::Relaxed);

        println!("Using result in guidance system: {}", result);
    }
}

fn main() {
    let guidance_system = Arc::new(GuidanceSystem {
        safe_mode: AtomicBool::new(false),
        unsafe_counter: AtomicUsize::new(0),
    });

    let guidance_system_thread = Arc::clone(&guidance_system);

    let handle = thread::spawn(move || {
        let payload_mass = 0.0000000001;
        let result = guidance_system_thread.safe_critical_calculation(payload_mass);
        println!("Result of safe critical calculation: {}", result);

        unsafe {
            guidance_system_thread.unsafe_use_result(result);
        }
    });

    let handles: Vec<_> = (0..4)
        .map(|_| {
            let guidance_system_thread = Arc::clone(&guidance_system);
            thread::spawn(move || {
                let payload_mass = 0.1;
                let result = guidance_system_thread.safe_critical_calculation(payload_mass);
                unsafe {
                    guidance_system_thread.unsafe_use_result(result);
                }
            })
        })
        .collect();

    handle.join().unwrap();

    for handle in handles {
        handle.join().unwrap();
    }

    println!(
        "Unsafe Counter Value: {}",
        guidance_system.unsafe_counter.load(Ordering::SeqCst)
    );
}

main()

Result of safe critical calculation: 2147483647
Using result in guidance system: 2147483647
Using result in guidance system: 3214442
Using result in guidance system: 3214442
Using result in guidance system: 3214442
Using result in guidance system: 3214442
Unsafe Counter Value: 2160341415


()

### Risks Associated with Common Unsafe Coding Practices

### Risk 1: Null Pointer Dereferencing

In [17]:
let null_pointer: *const i32 = std::ptr::null();

unsafe {
    let value = *null_pointer;
    println!("Dereferenced value: {}", value);
}

Dereferenced value: 21


()

### Risk 2: Buffer Overflows

In [18]:
let data: [u8; 5] = [1, 2, 3, 4, 5];

unsafe {
    let value = *data.get_unchecked(10);
    println!("Value obtained from buffer overflow: {}", value);
}

Value obtained from buffer overflow: 21


()

### Risk 3: Use-After-Free Errors

In [19]:
let data = Box::new(42);

let raw_pointer = Box::into_raw(data);

unsafe {
    drop(Box::from_raw(raw_pointer));
    println!("Value at raw pointer: {}", *raw_pointer);
}

Value at raw pointer: 524723331


()

---
---